# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = './Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

# imports
import pandas as pd
import os
import numpy as np
import warnings

# ignore annoying warnings
warnings.filterwarnings('ignore')

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Assumptions

### field names
- The fields "National" in SL and Liberia mean the same as the "Totals" field in Guinea, i.e. the Total number of cases in the country.

### missing data
- When there's a NaN (the value was not filled), we assume that the data was lost, or not written down.
- When a report (csv file) for a given day is missing, we assume that nothing happened during this day.
- We are only interested in the Totals/National columns. If a row contains a NaN in this column, we chose to discard the row. This should not be problematic (if the number of NaNs in *Totals* is reasonably low), since we consider the daily **average** per month.

### fields meaning and aggregation
- Since we don't have additionnal information on the data, especially about the meanings of some fields, we decide to compute the new cases separaterly for the confirmed, probable and suspect cases. Same goes for the deaths.
    - We also add a column that sums all the new cases/deaths


### Used/Discarded fields
- We only consider the totals/national fields, not the values per district/region. Sometimes, the sum of the values per region is a bit higher than the total. We assume that's because some smaller regions are not shown in the data.
    - For this reason, if the *Totals* value is NaN, we cannot just sum up the values of the different regions.
- After observing the data, we can safely assume that the deaths, unlike the new cases, are reported cumulatively. I.e., the number of deaths reported on day *d* is the number of deaths that happened this day, plus all the deaths that happened in the previous days. We did not use the *new deaths* values, because for some countries there is no distinction between the different cases (confirmed, probable and suspected).

## Overall Process
- Importing the data
- Merge and clean the full dataset
- Unstack Description column
- Daily averages computation
- Results
- Discussion

### 1. Importing the data

In [ ]:
# import data from CSVs
guinea = []
liberia = []
sl = []
dfs = [guinea, liberia, sl]
i = 0
folders = ["guinea_data", "liberia_data", "sl_data"]
for folder in folders:
    path = DATA_FOLDER + "/ebola/"
    for file in os.listdir(path + folder):
        dfs[i].append(pd.read_csv(path + folder + '/' + file))
    i += 1

In [ ]:
# import Guinea data into DataFrame
guinea = pd.DataFrame()
i = 0
for df in dfs[0]:
    df['Country'] = 'Guinea' # set the new country column
    dfs[0][i] = df
    i += 1
guinea = pd.concat(dfs[0])
guinea.head(2)

In [ ]:
# import liberia data into DataFrame
liberia = pd.DataFrame()
i = 0
for df in dfs[1]:
    df['Country'] = 'Liberia'
    df = df.rename(columns = {'National':'Totals', 'Variable':'Description'})
    dfs[1][i] = df
    i += 1
liberia = pd.concat(dfs[1])
liberia.head(2)

In [ ]:
# import Sierra Leone data into DataFrame
sl = pd.DataFrame()
i = 0
for df in dfs[2]:
    df['Country'] = 'Sierra Leone'
    df = df.rename(columns = {'National':'Totals', 'date' : 'Date', 'variable':'Description'})
    dfs[2][i] = df
    i += 1
sl = pd.concat(dfs[2])
sl.head(2)

### 2. Merge and clean the full dataset (all countries)

In [ ]:
# merge dataframes
full_data = pd.concat([guinea, liberia, sl])

# remove each row whose 'Totals' value is NaN
n_col_before = full_data.shape[0]
full_data = full_data[full_data.Totals.notnull()]
n_col_after = full_data.shape[0]
print("Removed " + str(n_col_before-n_col_after) + " rows out of " + str(n_col_before))

# Specify that the Date column contains datetime objects
full_data.Date = pd.to_datetime(full_data.Date)

# To facilitate readability, we discard all the columns we don't need
full_data_crop = full_data[['Country', 'Description', 'Totals', 'Date']]

This is what the dataframe that we'll use to compute the different values looks like so far: 

In [ ]:
 full_data_crop.head(2)

### 3. Unstack *Description* column

Each distinct value of the *Description* column becomes a new column. In order to do that, we first clean the different *Description* values so that there are consistent across the different countries.

For example, in Sierra Leone, The number of confirmed deaths is referred as *death_confirmed*, but as *Total death/s in confirmed cases* in Liberia.

Then, since we are only interested in deaths and new cases, we discard all the rows that concern other things.

Finally, we noticed when trying to unstack the data that there were three exact duplicate rows from the liberian data. Thus, we removed those rows.

We can then finally unstack the descriptions.

In [ ]:
# Names and regexes that we use to process the Description

interests_rgx = ['(new_suspected|New cases of suspects|New Case\/s \(Suspected\))',
             '(new_probable|New cases of probables|New Case\/s \(Probable\))',
             '(new_confirmed|New cases of confirmed|New case\/s \(confirmed\))',
             '(death_suspected|Total deaths of suspects|Total death\/s in suspected cases)',
             '(death_probable|Total deaths of probables|Total death\/s in probable cases)',
             '(death_confirmed|Total deaths of confirmed|Total death\/s in confirmed cases)'
            ]

interests_new_cases = ['New cases (suspected)', 'New cases (probable)', 'New cases (confirmed)']
interests_death = ['Deaths in suspected cases', 'Deaths in probable cases', 'Deaths in confirmed cases']
interests_names = interests_new_cases + interests_death

In [ ]:
# We decided to unstack the Descriptions column in other columns

# uniformize description in needed columns
for i in range(len(interests_rgx)):
    full_data_crop.Description = full_data_crop.Description.str.replace(interests_rgx[i], interests_names[i])

# remove un-needed rows based on Description column
_filter = full_data_crop.Description.isin(interests_names)
full_data_crop = full_data_crop[_filter]

# unstack data
full_data_crop.index = pd.MultiIndex.from_arrays([full_data_crop.Description.values, full_data_crop.Country.values, full_data_crop.Date.values])

# We discovered that there are 3 duplicated lines in the file 2014-10-04 from liberia
# We then had to delete the duplicated rows from the data so that we could unstack it
full_data_crop = full_data_crop[~full_data_crop.index.duplicated(keep='first')]
full_data_crop = full_data_crop.drop(['Description', 'Date', 'Country'], axis=1)

# unstack the Descriptions and remove unneccessary index level
full_data_crop = full_data_crop.unstack(level=0)
full_data_crop.columns = full_data_crop.columns.droplevel()

# convert fields to numeric values
full_data_crop = full_data_crop.apply(pd.to_numeric)

Here is what the data looks like at this point. We have a single dataframe, containing the 3 countries' data, from which we can compute the daily averages per month.

In [ ]:
full_data_crop

### 4. Daily averages computation

We defined two functions that, given the country name and names of columns we are interested in, return the daily average number of new cases (resp. deaths) per month in that country.

Since we have no further information about the meanings of *probable*, *suspected* and *confirmed*, we chose not to aggregate those values and compute a daily average for each of them.

Since we assumed deaths to be cumulative (see *Assumptions*), we had to find a different way to compute the average deaths. In order to do so, for each report we compute the difference between its *deaths* value and the *deaths* value of the preceding report. Then we compute the daily average per month from this difference.

In [ ]:
# returns a df containing the mean number of new cases for the country given in argument, 
# respectively for confirmed, probable and suspected new cases
def compute_new_cases(country, col_names):
    # take only the data for the country given in argument, and group the values by month
    country_data = full_data_crop[full_data_crop.Country == country]
    country_data = country_data.groupby(country_data.Date.dt.month)
    
    df = pd.DataFrame()
    for name in col_names:
        df[name] = country_data[name].mean()
    df['Country'] = country
    df = df.set_index(['Country'], append=True)
    df = df.swaplevel(0, 1)
    return df

# Same thing as the previous function, but for the number of deaths
def compute_deaths(country, col_names):
    # take only the data for the country given in argument, and group the values by month
    country_data = full_data_crop[full_data_crop.Country == country]
    
    df = pd.DataFrame()
    for name in col_names:
        death_values = country_data[name].subtract(country_data[name].shift(1))
        country_data[name] = death_values
    
    country_data = country_data.groupby(country_data.Date.dt.month)
    
    for name in col_names:
        df[name] = country_data[name].mean()
        #df[name] = (np.max(country_data[name]) - np.min(country_data[name])).divide((np.max(country_data['Date']) - np.min(country_data['Date'])).dt.days)
    df['Country'] = country
    df = df.set_index(['Country'], append=True)
    df = df.swaplevel(0, 1)
    return df

In [ ]:
# put Date and Country back as columns (from the indexes) to make groupbys cleaner
full_data_crop['Date'] = full_data_crop.index.get_level_values(1).values
full_data_crop['Country'] = full_data_crop.index.get_level_values(0).values

guinea_new_cases = compute_new_cases('Guinea', interests_new_cases)
guinea_deaths = compute_deaths('Guinea', interests_death)

liberia_new_cases = compute_new_cases('Liberia', interests_new_cases)
liberia_deaths = compute_deaths('Liberia', interests_death)

sl_new_cases = compute_new_cases('Sierra Leone', interests_new_cases)
sl_deaths = compute_deaths('Sierra Leone', interests_death)

new_cases_daily_means = pd.concat([guinea_new_cases, liberia_new_cases, sl_new_cases])
new_cases_daily_means['Total new cases'] = new_cases_daily_means.sum(axis=1)
deaths_daily_means = pd.concat([guinea_deaths, liberia_deaths, sl_deaths])
deaths_daily_means['Total deaths'] = deaths_daily_means.sum(axis=1)

### 5. Results

This is the daily average per month of new cases:

In [ ]:
deaths_daily_means.round(2)

This is the daily average per month of deaths:

In [ ]:
deaths_daily_means.round(2)

### 6. Discussion

#### NaN values
There are NaN values at some places in the results. This happens when there is not a single data sample for an entire month. We decided to leave these NaN values as is, as we don't have enough data to interpolate missing values. We just replaced the NaNs by "-" to make it clear that there is no data here.

#### Outliers
The only big outlier values we have are for Liberia (new cases) during the month of december. We suspect that at some point during the month, the reports became cumulative for some reason. Since we have no way to confirm it, we decided to leave these values as they are, although we could have processed them as cumulative values (as we did it for deaths).

#### Cumulative values decreasing
Sometimes, even though the values are cumulative, we noticed that some values can decrease over time. After some research on the internet, we found that it could be because of some mistakes that were corrected (people we thought were dead but actually no).

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
dfs = []
path = DATA_FOLDER + "/microbiome/"
for file in os.listdir(path):
    if file != 'metadata.xls' :
        dfs.append(pd.read_excel(path + '/' + file , header = None , index_col = 0))
    
Meta = pd.read_excel("Data/microbiome/metadata.xls", index_col = 0)    

In [ ]:
Meta['SAMPLE'].fillna('unknown' , inplace = True )
arrays = [Meta['GROUP'].values, Meta['SAMPLE'].values]
arrays

In [ ]:
merge = pd.concat(dfs , axis = 1)
merge.columns = arrays
merge.fillna('unknown' , inplace = True )
merge

In [ ]:
print(merge.shape)
merge.index.is_unique

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os  

df3 = pd.read_excel(DATA_FOLDER+'/titanic.xls')
HTML(filename=DATA_FOLDER+'/titanic.html')


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
df3.head()

### 1. Describing the type of attributes

In [ ]:
df3.dtypes

In [ ]:
df3.count()

From the information about the attributes of our dataset, we will set to caterogical (sex,cabin,embarked,boat and home.dest) features to categorical. Body, ticket and name normally being different for all passengers are not converted to categorical. 

There are 295 cabins

In [ ]:
print(np.shape(df3.groupby('name').count()))
df3.groupby('name').count()
a = df3.index[df3.name.isnull() == True]
df3.loc[a]
print((df3.name.value_counts()>1)[:5])

Here we can see that there are two people without names. when trying to see their attributes we see no rows. Maybe some people have their name written twice. Those people are Kelly, Mr. James and Connolly, Miss. Kate. Looking at their attributes, we will remove the versions with the least amount of data, loc[726,925]. We could also have decided to keep the four rows or to delete two of them in function of the fare they paid

In [ ]:
df3[df3.name.isin(['Connolly, Miss. Kate','Kelly, Mr. James'])]

In [ ]:
df3.drop(df3.index[[726,925]])

In [ ]:
#convert to categorical
df3["sex"] = pd.Categorical(df3['sex'], df3['sex'].unique())
df3['cabin'] = df3['cabin'].fillna('missing')
df3["cabin"] = pd.Categorical(df3['cabin'], df3['cabin'].unique())
df3['embarked'] = df3['embarked'].fillna('missing')
df3["embarked"] = pd.Categorical(df3['embarked'], df3['embarked'].unique())
df3['boat'] = df3['boat'].fillna('missing')
df3["boat"] = pd.Categorical(df3['boat'], df3['boat'].unique())
df3['home.dest'] = df3['home.dest'].fillna('missing')
df3["home.dest"] = pd.Categorical(df3['home.dest'], df3['home.dest'].unique())

df3.dtypes

In [ ]:
# Write your answer here
df3.describe()

talk about range and type

In [ ]:
df3.count()

now all the missing values have been cleaned except for the body column. Since there are only 121 entries out of 1309, we will see later if we remove this column.

### 2. Plot histograms

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(20,20)

df_temporary = df3.copy()
df_temporary.age = pd.cut(df3.age, [0,10,20,30,40,50,60,70,80,90])
sns.countplot(x='age',data=df_temporary,ax=axes[0,0])
axes[0,0].set_xlabel('age')
axes[0,0].set_ylabel('count')

classes = df3.groupby('pclass').size()
#df3.hist('pclass',ax=axes[0,1])
classes.plot(kind='bar',ax=axes[0,1])
axes[0,1].set_xlabel('class')
axes[0,1].set_ylabel('count')

sns.countplot(x='sex',data=df3,ax=axes[1,0])

sns.countplot(x='embarked',data=df3)

### Histogram observations
- We can observe the passenger age distribution in the titanic is gaussian
- There is a higher amount of males onboard
- There were more people embarked in third class then first and second combined
- Most of the passengers boarded in Southampton

### 3. Calculate the proportion of passengers by cabin floor
We make the assumption that for cabins with name as 'F E69' belong to the cabin floor F. It should be noted that the following pie chart does not represent perfectly the true proportion of passengers per cabin floor as only 295 out of 1309 values of the cabin column are filled.

In [ ]:
df3['cabin_floor'] = df3.cabin.apply(lambda x: x[0])
df3.cabin_floor = df3.cabin_floor[df3.cabin_floor != 'm']

In [ ]:
#plt.pie(df3.groupby('cabin_floor').count())
import plotly as ip
#plt.pie(df3.groupby('cabin_floor').count())
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
#ip.tools.set_credentials_file(username='USERNAME', api_key='KEY')
#trace = go.Pie(labels=df3['cabin_floor'].value_counts().index, values=df3['cabin_floor'].value_counts())
#py.iplot([trace], filename='Proportion of passengers per cabin floor')

#Image(url= "https://plot.ly/~mgelsm/2.embed")
from IPython.display import IFrame

IFrame('https://plot.ly/~mgelsm/2', width=800, height=600)
#![Image of Yaktocat](https://plot.ly/~mgelsm/2)

This image is plotted in case the viewer does not run the boxes. The interactive plot is however not available below.
<div>
    <a href="https://plot.ly/~mgelsm/2/?share_key=LTW3TPD50iXkdYukf55G9j" target="_blank" title="Proportion of passengers per cabin floor" style="display: block; text-align: center;"><img src="https://plot.ly/~mgelsm/2.png?share_key=LTW3TPD50iXkdYukf55G9j" alt="Proportion of passengers per cabin floor" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="mgelsm:2" sharekey-plotly="LTW3TPD50iXkdYukf55G9j" src="https://plot.ly/embed.js" async></script>
</div>

### 4. For each travel class, calculate the proportion of the passengers that survived. Present your results in pie charts.

In [ ]:
fig = ip.tools.make_subplots(rows=3, cols=1)
trace1 = go.Pie(labels=df3.loc[df3.pclass == 1].survived.value_counts().index, values=df3.loc[df3.pclass == 1].survived.value_counts())
fig.append_trace(trace1, 1, 1)
trace2 = go.Pie(labels=df3.loc[df3.pclass == 2].survived.value_counts().index, values=df3.loc[df3.pclass == 2].survived.value_counts())
fig.append_trace(trace2, 2, 1)
trace1 = go.Pie(labels=df3.loc[df3.pclass == 3].survived.value_counts().index, values=df3.loc[df3.pclass == 1].survived.value_counts())
fig.append_trace(trace3, 3, 1)
py.plot(fig)

### 5. Calculate the proportion of the passengers that survived by travel class and sex. Present your results in a single histogram

In [ ]:
import seaborn as sns
sns.barplot(x='sex',y='survived',hue='pclass',data=df3)

Here we can see that more women survived from the titanic sinking and upper classes have a higher rate of survival.

### 6. Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. Present your results in a DataFrame with unique index.

In [ ]:
df3['age_class'] = df3.age < df3.age.median()
df3.age.median()

In [ ]:
def create_category(x):
    if x.age_class == False and x.sex == 'female':
        val = 'young_female'
    elif x.age_class == True and x.sex == 'female':
        val = 'old_female'
    elif x.age_class == False and x.sex == 'male':
        val = 'young_male'
    else:
        val = 'old_male'   
    return val
            
    

In [ ]:
df3['age_sex'] = df3.apply(create_category,axis=1)

In [ ]:
sns.barplot(x='age_sex',y='survived',hue='pclass',data=df3)

This histogram shows two important things :
1. There is a higher rate of Women and children that survived
2. People in hihger classes had better chances of surviving

Hence we can understand that the rule of saving children and women first was applied during the titanic sinking.